In [10]:
import pandas as pd

get_data = lambda x, y, n: [(x, y) * 2 for _ in range(n)]
data = get_data(1, 1, 207) + get_data(1, 0, 282) + get_data(0, 1, 231) + get_data(0, 0, 242)
pdf = pd.DataFrame(data, columns=['x1', 'x2', 'x3', 'x4'])
sdf = sqlContext.createDataFrame(pdf)

In [13]:
sdf.printSchema()

root
 |-- x1: long (nullable = true)
 |-- x2: long (nullable = true)
 |-- x3: long (nullable = true)
 |-- x4: long (nullable = true)



In [29]:
from itertools import combinations

def to_counts(d):
    def as_key(k1, k2):
        keys = sorted([k1, k2])
        return keys[0], keys[1]
    
    def as_count(v1, v2):
        a, b, c, d = 0, 0, 0, 0
        if v1 is not None and v2 is not None:
            if v1 == 1 and v2 == 1:
                a = 1
            elif v1 == 1 and v2 == 0:
                b = 1
            elif v1 == 0 and v2 == 1:
                c = 1
            else:
                d = 1
        return a, b, c, d
    
    def transform(k1, k2):
        v1, v2 = d[k1], d[k2]
        return as_key(k1, k2), as_count(v1, v2)
    
    return [transform(k1, k2) for k1, k2 in combinations(d.keys(), 2)]

def add_counts(a, b):
    return a[0] + b[0], a[1] + b[1], a[2] + b[2], a[3] + b[3]

# to_counts({'x1': 1, 'x2': 1, 'x3': 1, 'x4': 1})

In [31]:
sdf.rdd\
    .flatMap(lambda r: to_counts(r.asDict()))\
    .reduceByKey(lambda a, b: add_counts(a, b))\
    .sortByKey()\
    .collect()

[(('x1', 'x2'), (207, 282, 231, 242)),
 (('x1', 'x3'), (489, 0, 0, 473)),
 (('x1', 'x4'), (207, 282, 231, 242)),
 (('x2', 'x3'), (207, 231, 282, 242)),
 (('x2', 'x4'), (438, 0, 0, 524)),
 (('x3', 'x4'), (207, 282, 231, 242))]